# Notebook: Limpieza y modelos XGBoost

Este notebook realiza:
- Limpieza robusta del dataset `Fleet_Work_Orders_20251111.csv`.
- Tres pipelines de modelado con XGBoost:
  1. Regresión para **`Total Cost`**.
  2. Clasificación para **`On Time Indicator`** (Y/N).
  3. Regresión para **`Days to next revision`** (heurística: tiempo hasta la siguiente orden del mismo vehículo).

Cada celda incluye explicación y pasos para que sea reproducible. Guarda el notebook y ejecútalo en el mismo folder donde está el CSV.


In [ ]:
# Instalar librerías necesarias (ejecutar solo si hace falta)
!pip install -q xgboost scikit-learn pandas matplotlib seaborn


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Ruta del CSV (debe estar en el mismo directorio que este notebook)
CSV_PATH = 'Fleet_Work_Orders_20251111.csv'
assert Path(CSV_PATH).exists(), f"No encontré {CSV_PATH} en el directorio actual."

# Cargar y mostrar primeras filas
df = pd.read_csv(CSV_PATH)
df.head()


In [ ]:
# Información rápida
df.info()
df.isna().sum()


## Limpieza y normalización
Convertiremos fechas, forcemos conversión numérica en columnas clave, limpiamos textos y creamos la variable "days to next revision".


In [ ]:
date_cols = [
    'In Service Date',
    'Work Order Begin Date',
    'Work Order Finish Date'
]
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors='coerce')

# Limpiar textos
text_cols = ['Vehicle Make', 'Vehicle Model', 'WO Reason']
for c in text_cols:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip().str.upper().replace({'NAN':'UNKNOWN'})

# Normalizar indicadores
if 'On Time Indicator' in df.columns:
    df['On Time Indicator'] = df['On Time Indicator'].astype(str).str.upper().str.strip()
    df['On Time Indicator'] = df['On Time Indicator'].replace({'YES':'Y','NO':'N','1':'Y','0':'N'})
    df['On Time Indicator'] = df['On Time Indicator'].replace({'NAN':'UNKNOWN'})

if 'Open Indicator' in df.columns:
    df['Open Indicator'] = df['Open Indicator'].astype(str).str.upper().str.strip()
    df['Open Indicator'] = df['Open Indicator'].replace({'YES':'Y','NO':'N','1':'Y','0':'N'})
    df['Open Indicator'] = df['Open Indicator'].replace({'NAN':'UNKNOWN'})

# Forzar y limpiar numéricos importantes
numeric_fix_cols = [
    'Total Cost',
    'Expected Hours',
    'Actual Hours',
    'Days to Complete',
    'WO Vehicle Odometer',
    'Vehicle Year'
]
for col in numeric_fix_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace(',', '', regex=False).str.replace('$','',regex=False).str.strip()
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Rellenar numéricos con 0 por defecto (puedes cambiar la estrategia)
for col in numeric_fix_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Eliminar duplicados
df = df.drop_duplicates()

# Mostrar resumen
df[numeric_fix_cols].dtypes


### Crear target: days to next revision
Calcularemos para cada vehículo el número de días hasta la siguiente orden: para cada grupo por `Vehicle Number`, ordenamos por `Work Order Begin Date` y calculamos la diferencia entre la siguiente `Work Order Begin Date` y la fila actual `Work Order Finish Date`. Si no existe siguiente orden, pondremos `NaN` y luego rellenaremos con 0 (o la mediana si prefieres).


In [ ]:
if 'Vehicle Number' in df.columns and 'Work Order Begin Date' in df.columns and 'Work Order Finish Date' in df.columns:
    df = df.sort_values(['Vehicle Number','Work Order Begin Date'])
    df['next_begin'] = df.groupby('Vehicle Number')['Work Order Begin Date'].shift(-1)
    df['days_to_next_revision'] = (df['next_begin'] - df['Work Order Finish Date']).dt.days
    # si negative o NaN -> rellenar con 0
    df['days_to_next_revision'] = df['days_to_next_revision'].apply(lambda x: x if pd.notna(x) and x>=0 else np.nan)
    df['days_to_next_revision'] = df['days_to_next_revision'].fillna(0)
else:
    df['days_to_next_revision'] = 0

df['days_to_next_revision'].head()


## Preparar features y targets para los tres problemas:
1. Regression -> `Total Cost`
2. Classification -> `On Time Indicator` (Y/N). Convertiremos a 1/0.
3. Regression -> `days_to_next_revision`


In [ ]:
# Definir features base
feature_cols = [
    'Total Cost',
    'Expected Hours',
    'Days to Complete',
    'WO Vehicle Odometer',
    'Vehicle Year',
    'Vehicle Make',
    'Vehicle Model',
    'WO Reason',
    'On Time Indicator',
    'Open Indicator'
]
# Filtrar las columnas que realmente existen
feature_cols = [c for c in feature_cols if c in df.columns]
feature_cols


In [ ]:
# Targets
y_cost = df['Total Cost']
y_time = df['Actual Hours']
y_next = df['days_to_next_revision']

# Features copy
X = df[feature_cols].copy()
X.shape


### Convertir categóricas a tipo `category` y asegurar que 'UNKNOWN' exista


In [ ]:
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
for c in cat_cols:
    X[c] = X[c].astype(str).str.upper().str.strip().replace({'NAN':'UNKNOWN'})
    X[c] = X[c].astype('category')
    if 'UNKNOWN' not in X[c].cat.categories:
        X[c] = X[c].cat.add_categories(['UNKNOWN'])

X.dtypes


In [ ]:
# Separar numéricas y categóricas (ahora categories están como 'category')
num_cols = X.select_dtypes(include=['int64', 'float64', 'Int64']).columns.tolist()
cat_cols = X.select_dtypes(include=['category']).columns.tolist()
num_cols, cat_cols


In [ ]:
# Rellenar NaNs: numéricos -> 0, categóricas -> 'UNKNOWN'
X.loc[:, num_cols] = X[num_cols].fillna(0)
for col in cat_cols:
    if 'UNKNOWN' not in X[col].cat.categories:
        X[col] = X[col].cat.add_categories(['UNKNOWN'])
X.loc[:, cat_cols] = X[cat_cols].fillna('UNKNOWN')

# Confirmar no hay NaNs
print('NaNs in X:', X.isna().sum().sum())
print('NaNs in y_time:', y_time.isna().sum())
print('NaNs in y_cost:', y_cost.isna().sum())
print('NaNs in y_next:', y_next.isna().sum())


## Modelado: XGBoost
Entrenaremos 3 modelos (2 regresores y 1 clasificador).

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, f1_score, classification_report
import numpy as np

def train_and_eval_reg(X, y, label='reg'):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, enable_categorical=True)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    print(f"{label} MAE: {mae:.4f} | RMSE: {rmse:.4f}")
    return model, X_test, y_test, preds

def train_and_eval_clf(X, y, label='clf'):
    # convertir target a 0/1 si viene como 'Y'/'N' o 'UNKNOWN'
    y2 = y.astype(str).str.upper().map({'Y':1, 'N':0})
    y2 = y2.fillna(0)
    X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.2, random_state=42)
    model = XGBClassifier(n_estimators=200, max_depth=6, enable_categorical=True)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds, zero_division=0)
    print(f"{label} ACC: {acc:.4f} | F1: {f1:.4f}")
    print(classification_report(y_test, preds, zero_division=0))
    return model, X_test, y_test, preds


### 1) Modelo para **Total Cost** (regresión)


In [ ]:
model_cost, Xc_test, yc_test, preds_cost = train_and_eval_reg(X, y_cost, label='TotalCost')


### 2) Modelo para **Actual Hours** (regresión)


In [ ]:
model_time, Xt_test, yt_test, preds_time = train_and_eval_reg(X, y_time, label='ActualHours')


### 3) Modelo para **Days to next revision** (regresión)


In [ ]:
model_next, Xn_test, yn_test, preds_next = train_and_eval_reg(X, y_next, label='DaysNext')


### 4) Clasificador para **On Time Indicator** (si prefieres clasificación)


In [ ]:
if 'On Time Indicator' in X.columns:
    model_on_time, Xot_test, yot_test, preds_ot = train_and_eval_clf(X, df['On Time Indicator'], label='OnTime')
else:
    print('No se encontró On Time Indicator en X')


## Guardar modelos
Guardamos los modelos entrenados para usarlos luego desde FastAPI.


In [ ]:
model_cost.save_model('model_cost.json')
model_time.save_model('model_time.json')
model_next.save_model('model_next.json')
if 'model_on_time' in globals():
    model_on_time.save_model('model_on_time.json')
print('Modelos guardados en el directorio actual')


## Ejemplo de uso desde FastAPI
Agrega estos snippets en tu `main.py` para exponer endpoints `/predict_cost`, `/predict_time`, `/predict_next` y `/predict_on_time`.


In [ ]:
%%bash
cat > fastapi_predict_snippets.txt <<'PY'
# --- snippets para main.py ---
import xgboost as xgb
import pandas as pd
from pydantic import BaseModel

# Cargar modelos (en el startup de FastAPI)
model_cost = xgb.XGBRegressor()
model_cost.load_model('model_cost.json')

model_time = xgb.XGBRegressor()
model_time.load_model('model_time.json')

model_next = xgb.XGBRegressor()
model_next.load_model('model_next.json')

model_on_time = None
try:
    model_on_time = xgb.XGBClassifier()
    model_on_time.load_model('model_on_time.json')
except Exception:
    pass

# Ejemplo de Pydantic para /predict_cost
class CostInput(BaseModel):
    Expected_Hours: float
    Actual_Hours: float
    Days_to_Completion: float
    WO_Vehicle_Odometer: float
    Vehicle_Year: int
    Vehicle_Make: str
    Vehicle_Model: str
    WO_Reason: str
    On_Time_Indicator: str
    Open_Indicator: str

# End of snippets
PY
echo 'Snippets guardados en fastapi_predict_snippets.txt'


### Fin del notebook
Guarda y ejecuta las celdas en orden. Si quieres, puedo ajustar hiperparámetros, crear pipelines con búsqueda (GridSearch / Optuna) o integrar estas predicciones directamente como endpoints en `main.py`.